In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#read excel file into pandas dataframe
df=pd.read_excel('t_19&20.xlsx',engine='openpyxl')



In [3]:

#Setting Predictors
X=df.drop(columns=['Datetime','Temp(F)']).values

#Setting Target
Y=df['Temp(F)'].values

#Splitting dataset into Train and Test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
print(X_test)

[[   9   16 2019    2    0    0]
 [  12    3 2019    6    0    0]
 [  10   30 2019   12    0    0]
 ...
 [   2    1 2019   20    0    0]
 [   5   30 2019    6    0    0]
 [  11   14 2019   12    0    0]]


In [4]:
# Building  Random Forest Classifier
from sklearn.ensemble import RandomForestRegressor
rfr_temp = RandomForestRegressor(random_state =42 )
rfr_temp.fit(X_train, Y_train)



RandomForestRegressor(random_state=42)

In [5]:
# Evaluating on Training set
rfr_pred_train = rfr_temp.predict(X_train)
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(Y_train,rfr_pred_train)))
print(r2_score(Y_train,rfr_pred_train))


# Evaluating on Test set
rfr_pred_test = rfr_temp.predict(X_test)
print(np.sqrt(mean_squared_error(Y_test,rfr_pred_test)))
print(r2_score(Y_test,rfr_pred_test))


1.1459808353046101
0.9956813024701605
3.0535080745872123
0.969574436848327


In [6]:

#Saving to S3
import tempfile
import joblib
import boto3
s3 = boto3.resource('s3')
bucket='digitwin-airqualitydata'
key='EPA_data/EPA_Temp_regress.sav'

# WRITE
with tempfile.TemporaryFile() as fp:
    joblib.dump(rfr_temp, fp)
    fp.seek(0)
    s3.Bucket(bucket).put_object(Key= key, Body=fp.read())